# Rapid API post

- title: Rapid API post
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [3]:
HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://airbnb19.p.rapidapi.com/api/v1/getLanguages"))
		.header("X-RapidAPI-Key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
		.header("X-RapidAPI-Host", "airbnb19.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

{"status":true,"message":"Success","timestamp":1664866535098,"data":[{"id":"ar-SA","title":"Arabic (Saudi Arabia)"},{"id":"zh-CN","title":"Chinese (China)"},{"id":"zh-HK","title":"Chinese (Hong Kong SAR China)"},{"id":"zh-TW","title":"Chinese (Taiwan)"},{"id":"cs-CZ","title":"Czech (Czech Republic)"},{"id":"da-DK","title":"Danish (Denmark)"},{"id":"nl-BE","title":"Dutch (Belgium)"},{"id":"nl-NL","title":"Dutch (Netherlands)"},{"id":"en-AU","title":"English (Australia)"},{"id":"en-IE","title":"English (Ireland)"},{"id":"en-ZA","title":"English (South Africa)"},{"id":"en-GB","title":"English (United Kingdom)"},{"id":"en-US","title":"English (United States)"},{"id":"fi-FI","title":"Finnish (Finland)"},{"id":"fr-CA","title":"French (Canada)"},{"id":"fr-FR","title":"French (France)"},{"id":"de-DE","title":"German (Germany)"},{"id":"el-GR","title":"Greek (Greece)"},{"id":"he-IL","title":"Hebrew (Israel)"},{"id":"hi-IN","title":"Hindi (India)"},{"id":"hu-HU","title":"Hungarian (Hungary)"},{"i

In [ ]:
package com.nighthawk.spring_portfolio.controllers;


import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.Date;
import java.util.HashMap;

import org.json.simple.parser.JSONParser;
import org.json.simple.JSONObject;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

@RestController
public class Coins {
    private JSONObject body; //last run result
    private HttpStatus status; //last run status
    String last_run = null; //last run day of month

    @GetMapping("/coins")
    public ResponseEntity<JSONObject> getCoins() {

        //calls API once a day, sets body and status properties
        String today = new Date().toString().substring(0,10); 
        if (last_run == null || !today.equals(last_run))
        {
            try {  //APIs can fail (ie Internet or Service down)
                
                //RapidAPI header
                /* 
                HttpRequest request = HttpRequest.newBuilder()
		        .uri(URI.create("https://coinranking1.p.rapidapi.com/coins?referenceCurrencyUuid=yhjMzLPhuIDl&timePeriod=24h&tiers%5B0%5D=1&orderBy=marketCap&orderDirection=desc&limit=50&offset=0"))
		        .header("X-RapidAPI-Key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
		        .header("X-RapidAPI-Host", "coinranking1.p.rapidapi.com")
		        .method("GET", HttpRequest.BodyPublishers.noBody())
		        .build();
                
                
                HttpRequest request = HttpRequest.newBuilder()
		        .uri(URI.create("https://wft-geo-db.p.rapidapi.com/v1/geo/cities"))
		        .header("X-RapidAPI-Key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
		        .header("X-RapidAPI-Host", "wft-geo-db.p.rapidapi.com")
		        .method("GET", HttpRequest.BodyPublishers.noBody())
		        .build();
                */

                
                HttpRequest request = HttpRequest.newBuilder()
		        .uri(URI.create("https://airbnb19.p.rapidapi.com/api/v1/getLanguages"))
		        .header("X-RapidAPI-Key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
		        .header("X-RapidAPI-Host", "airbnb19.p.rapidapi.com")
		        .method("GET", HttpRequest.BodyPublishers.noBody())
		        .build();
               
                HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
                //System.out.println(response.body());

                //System.out.println(response.body());
                //JSONParser extracts text body and parses to JSONObject
                //this.body = (JSONObject) new JSONParser().parse(response.body());
                JSONParser parser = new JSONParser();  
                this.body = (JSONObject) parser.parse(response.body());   
                this.status = HttpStatus.OK;  //200 success
                this.last_run = today;
                
            }
            catch (Exception e) {  //capture failure info
                HashMap<String, String> status = new HashMap<>();
                status.put("status", "RapidApi failure: " + e);

                //Setup object for error
                this.body = new JSONObject(status);
                this.status = HttpStatus.INTERNAL_SERVER_ERROR; //500 error
                this.last_run = null;
            }
        }

        //return JSONObject in RESTful style
        return new ResponseEntity<>(body, status);
    }
}

### Challenges

Some challenges I encountered along the way was in the JSON parser and JSONObject line. This line parses JSONObject data and not any other kind. Initially, my data was in a JSONArray and to convert into JSONObject is not possible, so to fix this error. I had to find a new rapid api which is a JSONObject rather than a JSONArray. 

For Reference: 
* JSONObject start with a curly braces {}
* JSONArray start with a sqaure brackets []

You can find these in the results page of the your rapid api, at the top.